In [1]:
from simple_net import SimpleNet, SimpleNetV2
from dataset import MurmurDataset
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch
from torch.utils.data import DataLoader
import time
from tqdm import tqdm

In [2]:
net = SimpleNetV2()
net(torch.ones(1,1,10_000)).shape

torch.Size([1, 32, 1])

In [7]:
model = SimpleNetV2()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))
model

132804


SimpleNetV2(
  (conv_1): Conv1d(1, 32, kernel_size=(32,), stride=(2,), padding=(15,), padding_mode=reflect)
  (norm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_1): GELU(approximate='none')
  (conv_2): Conv1d(32, 32, kernel_size=(32,), stride=(2,), padding=(15,), padding_mode=reflect)
  (norm_2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_2): GELU(approximate='none')
  (conv_3): Conv1d(32, 64, kernel_size=(32,), stride=(2,), padding=(15,), padding_mode=reflect)
  (norm_3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_3): GELU(approximate='none')
  (conv_4): Conv1d(64, 1, kernel_size=(1,), stride=(1,), padding_mode=reflect)
  (norm_4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_4): GELU(approximate='none')
  (pool): AdaptiveAvgPool1d(output_size=256)
  (fc_1): Linear(in_features=256, out_features=128, bias=True)
  

In [3]:
num_epochs = 10
batch_size = 32
learning_rate = 1e-3

dataset = MurmurDataset()

In [4]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

model = SimpleNet()
criterion = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-5)

model.train()
for epoch in range(num_epochs):
    start = time.time()

    for i, data in tqdm(enumerate(dataloader)):
        # import pdb; pdb.set_trace()
        sample = data[0].unsqueeze(1)
        sample = Variable(sample).float()

        expected_output= data[1].float()
        output = model(sample).squeeze(1,2)
        # import pdb;pdb.set_trace()
        loss = criterion(output, expected_output)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch == 0:
        time_diff = time.time() - start
        print(f'Epoch took {round(time_diff, 4)}s')
        print(f'Expected hours to train: {time_diff * num_epochs / 3600 }')
    if epoch % 5 == 0:
        print(f'epoch [{epoch+1}/{num_epochs}]')
        print(round(loss.item(), 5))

533it [02:05,  4.23it/s]


Epoch took 125.9249s
Expected hours to train: 0.34979152745670744
epoch [1/10]
0.26562


533it [02:12,  4.01it/s]
533it [02:23,  3.70it/s]
533it [02:32,  3.51it/s]
533it [02:22,  3.74it/s]
533it [02:41,  3.30it/s]


epoch [6/10]
0.24741


533it [02:39,  3.34it/s]
533it [02:46,  3.21it/s]
533it [02:48,  3.17it/s]
533it [02:52,  3.09it/s]


In [5]:
torch.save(model, 'simple.pt')

In [6]:
sample

tensor([[[ 4.1921e-06,  2.3549e-06,  4.4300e-07,  ...,  7.8026e-04,
          -1.8907e-05, -1.2730e-03]],

        [[ 2.6557e-04,  2.4906e-04,  2.3256e-04,  ...,  3.7126e-03,
          -1.1970e-02, -1.8024e-02]],

        [[-2.7016e-04, -1.3597e-04,  1.9680e-04,  ...,  1.5009e-05,
          -3.0702e-06, -1.0368e-05]],

        ...,

        [[-4.5361e-06,  4.7199e-04,  9.7559e-04,  ..., -7.9938e-03,
          -3.4422e-03, -2.0001e-03]],

        [[-2.0687e-07, -9.1591e-07, -1.5288e-06,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.5652e-03,  2.5242e-03,  1.1701e-03,  ...,  8.7073e-05,
          -2.2999e-03, -3.1000e-03]]])

In [7]:
expected_output

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1.])

Output

In [9]:
model(sample).squeeze(1,2)

tensor([0.5449, 1.0000, 0.0969, 1.0000, 0.0039, 0.1587, 0.0327, 0.9995, 0.0018,
        0.9470, 0.0392, 1.0000, 0.0353, 0.2567, 0.0385, 0.9989, 0.0378, 0.9992,
        0.0384, 0.9965], grad_fn=<SqueezeBackward2>)

In [10]:
sample.shape

torch.Size([20, 1, 10000])